In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Pre-Processing

In [83]:
netflix = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/netflix_final_data.csv')
ratings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/title.ratings.tsv', sep='\t')
basics = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/title.basics.tsv', sep='\t', usecols=['tconst', 'titleType', 'isAdult', 'runtimeMinutes', 'genres'])

/var/folders/cl/5v9cpvt93sjd32_z1w8t6b0c0000gn/T/ipykernel_21758/1367458723.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/title.basics.tsv', sep='\t', usecols=['tconst', 'titleType', 'isAdult', 'runtimeMinutes', 'genres'])


### Merging and Cleaning Further

In [84]:
df = netflix.merge(ratings, how='left', on='tconst').merge(basics, how='left', on='tconst', )

In [85]:
#My Wonderful Life Manually added data

df.loc[df['title']=='my wonderful life', 'averageRating'] = 5.9
df.loc[df['title']=='my wonderful life', 'numVotes'] = 714
df.loc[df['title']=='my wonderful life', 'titleType'] = 'movie'
df.loc[df['title']=='my wonderful life', 'runtimeMinutes'] = 99
df.loc[df['title']=='my wonderful life', 'isAdult'] = 0
df.loc[df['title']=='my wonderful life', 'genres'] = 'Comedy,Drama,Romance'

### Change isTop10 Column into Integers and Replace \\N into NaN

In [86]:
df['is_top10'] = df['is_top10'].astype(int)
df.replace('\\N', np.nan, inplace=True)

### Split into Movies and Shows

In [87]:
movies, tv_shows = df[df['is_tv_show']==False], df[df['is_tv_show']==True]

## MOVIES

### Drop Columns that aren't Needed

In [88]:
# drop columns that are not needed
movies = movies.drop(['tconst', 'combined_title', 'title', 'season', 'is_tv_show', 'isAdult'], axis = 1)

In [89]:
from sklearn.model_selection import train_test_split
movies_train, movies_test = train_test_split(movies, test_size=0.3, random_state=100)

# Training set
movies_train_X = movies_train.drop(['is_top10'], axis = 1)
movies_train_Y = movies_train['is_top10']

# Test set
movies_test_X = movies_test.drop(['is_top10'], axis = 1)
movies_test_Y = movies_test['is_top10']

In [90]:
# remove any rows with NaN genres and titleType because those won't be handled by the transformer
# training set removal
indexRowsToRemove_train = movies_train_X[(movies_train_X.genres.isna()) & (movies_train_X.titleType.isna())].index
movies_train_X.drop(index = indexRowsToRemove_train[0], axis = 1, inplace = True)
movies_train_Y.drop(index = indexRowsToRemove_train[0], axis = 1, inplace = True)

# test set removal
indexRowsToRemove_test = movies_test_X[(movies_test_X.genres.isna()) & (movies_test_X.titleType.isna())].index
movies_test_X.drop(index = indexRowsToRemove_test[0], axis = 1, inplace = True)
movies_test_Y.drop(index = indexRowsToRemove_test[0], axis = 1, inplace = True)

In [91]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer

In [92]:
movies_test_X.isna().sum()

release_year_x     0
averageRating      3
numVotes           3
titleType          0
runtimeMinutes    13
genres             0
dtype: int64

In [93]:
transformer = ColumnTransformer(
    [("onehot",OneHotEncoder(drop='first', handle_unknown='ignore'), ['titleType']),                # One hot encode title types
    ("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),   # Impute missing ratings, votes, and runtime with the average
    ("bagofw", CountVectorizer(), 'genres')                                                         # Splits genres into bag of words and encode them
    ],
    remainder='passthrough')

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import sklearn.preprocessing
import sklearn.linear_model

# make sklearn pipeline with transform, normalization, and model
pipe = Pipeline(
    steps=[
        ('transformer', transformer),                                                       # Transform dataset                
        ('normalizer', sklearn.preprocessing.Normalizer()),                                 # Normalise features
        ('standardizer', sklearn.preprocessing.StandardScaler(with_mean=False)),            # Rescale dataset
        ('logistic_regress', sklearn.linear_model.LogisticRegression(max_iter = 1000))      # Build logistic Regression model
        
    ]
)

pipe2 = Pipeline(
    steps=[
        ('transformer', transformer)]
)     

In [95]:
# use grid search to cross validate logistic regression model - 5-fold cross validation
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(
    pipe,
    param_grid={
        'logistic_regress__C': [0.01, 0.1, 1, 10, 100],
        'logistic_regress__solver': ['lbfgs','saga'],
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [96]:
gs.fit(movies_train_X, movies_train_Y)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/s

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('imputer',
                                                                         SimpleImputer(),
                                                                         ['averageRating',
                                                                          'numVotes',
                                                                          'runtimeMinutes']),
            